# Notebook Setup

### Install Packages and Mount Drive

In [ ]:
%%capture
!pip install folium geopandas rasterio rasterstats

In [ ]:
import os
from google.colab import drive

# mount to google drive
drive.mount('/content/drive', force_remount=True)

# set working directories
working_dir = r'/content/drive/Shared drives/Regen - Science/Sams Random Stuff/Extract Spectral Values Example/'

Mounted at /content/drive


### Setup Basemap
Used for plotting purposes to help visualize the example

In [ ]:
import folium
import pyproj

# project center 
project_centroid = field_boundary.geometry[0].centroid
field_center = [project_centroid.y, project_centroid.x]

def get_basemap(map_center=field_center, zoom_start=13):
    return folium.Map(location=map_center, 
               zoom_start=zoom_start, 
               tiles='https://api.mapbox.com/v4/mapbox.satellite/{z}/{x}/{y}.png?access_token=pk.eyJ1Ijoib3Vyc2NpIiwiYSI6ImNqb2ljdHMxYjA1bDAzcW03Zjd0cHBsbXMifQ.rL9QPLvi0kLP3DzLt1PQBA',
               attr='map copyright Mapbox',
    )

def get_bbox(bounds, s_srs, t_srs='epsg:4326'):
    project = pyproj.Transformer.from_crs(
        pyproj.CRS(s_srs),
        pyproj.CRS(t_srs),
    ).transform

    src_x, src_y = [bounds.left, bounds.right], [bounds.bottom, bounds.top]
    tgt_x, tgt_y = project(src_x, src_y)
    bbox = [[tgt_x[0], tgt_y[0]], [tgt_x[1], tgt_y[1]]]

    return bbox

### Import Sampling Locations

In [ ]:
import geopandas as gpd

# sample locations (input file types could be geojson, geopackage, shapefile, etc...)
sample_locations_geojson = os.path.join(working_dir, 'sample_locations.geojson')
sample_locations = gpd.read_file(sample_locations_geojson)

# field boundary (for mapping & reference)
field_boundary_geojson = os.path.join(working_dir, 'field_boundary.geojson')
field_boundary = gpd.read_file(field_boundary_geojson)

# the files are read into a pd DataFrame so you can use any pandas operation
sample_locations.head()

,fid,SOC,Sample Location,geometry
0,1,2.73,SW03B,POINT Z (415907.004 7113403.811 177.730)
1,2,3.02,W12A,POINT Z (416506.021 7118415.894 0.000)
2,3,2.79,W11C,POINT Z (416772.825 7118151.784 0.000)
3,4,2.02,W14C,POINT Z (418134.709 7117944.425 0.000)
4,5,3.83,W20E,POINT Z (421670.482 7117499.113 0.000)


In [ ]:
# project boundary
overview_map = get_basemap(field_center)

boundary_layer = folium.GeoJson(data=field_boundary,
                        style_function=lambda x: {'fillColor': '#00000000'}, 
                        name='Project Boundary').add_to(overview_map)

# sample locations
sample_locations_layer = folium.GeoJson(data=sample_locations,
                                        tooltip=folium.GeoJsonTooltip(['SOC']),
                                        name='2019 Soil Samples').add_to(overview_map)

overview_map.add_child(folium.LayerControl())
overview_map

### Import Raster Layer
In this example the raster layer is a land cover classification

In [ ]:
import rasterio as rio

land_cover_tiff = os.path.join(working_dir, 'land_cover_classification.tif')
with rio.open(land_cover_tiff) as src:
    img_arr = src.read(1)
    img_meta = src.meta
    img_affine = src.transform
    img_bbox = get_bbox(src.bounds, src.crs.to_epsg())

img_meta

{'count': 1,
 'crs': CRS.from_epsg(32756),
 'driver': 'GTiff',
 'dtype': 'int16',
 'height': 1059,
 'nodata': -999.0,
 'transform': Affine(10.0, 0.0, 413240.0,
       0.0, -10.0, 7120890.0),
 'width': 1458}

In [ ]:
img_cmap = {1: (57, 177, 85, 1), #green = forest
            2: (69, 135, 242, 1), # blue = water
            3: (242, 175, 69, 1), # yellow = grass / pasture
            4: (162, 157, 148, 1), #grey = manmade
 }

basemap = get_basemap()
land_cover_layer = folium.FeatureGroup(name='Land Cover Classification')
land_cover_layer.add_child(folium.raster_layers.ImageOverlay(
                            img_arr, 
                            img_bbox,
                            colormap=lambda x: img_cmap[x],
                            opacity=0.5
                            ))
land_cover_layer.add_to(basemap)
basemap.add_child(folium.LayerControl())
basemap

# Sample Raster Values

In [ ]:
import pandas as pd
from rasterstats import point_query 

# helper function to get the image value at thepoint location
def extract_single_value(point, raster, band):
    return point_query(point, raster, band, interpolate='nearest')[0]


def sample_raster_value(raster, raster_meta, point_locations):
    # some images might have more than one band
    band_count = img_meta['count']
    band_names = ['band_1'+str(i+1) for i in range(band_count)]

    point_locations["key"] = point_locations.index
    points_df = point_locations
    
    # reproject sampling points to match input raster
    if not img_meta['crs'] == points_df.crs:
        points_df = points_df.to_crs(img_meta['crs'])
    
    extracted_values = []
    for index, row in points_df.iterrows():
        # extract sample location - hacky solution to deal with different types of point files
        if row.geometry.geom_type == "MultiPoint":
            sample_location = row.geometry[0]
        if row.geometry.geom_type == "Point":
            sample_location = row.geometry
        
        # extract spectral value at the sample location
        r_vals = [extract_single_value(sample_location, raster, i+1) for i in range(band_count)]
        extracted_values.append([row.key] + r_vals)
    
    # merge datasets
    cols = ['key'] + band_names
    svals_df = pd.DataFrame(extracted_values, columns=cols)
    point_locations = point_locations.merge(svals_df, on='key')

    return point_locations.drop(['key'], axis=1)

In [ ]:
extracted_values_df = sample_raster_value(land_cover_tiff, img_meta, sample_locations)

The function above extracts the image values at point locations and stores them in a geopandas dataframe. In this example band_1 indicates the land classification value. 

In [ ]:
extracted_values_df

,fid,SOC,Sample Location,geometry,band_11
0,1,2.73,SW03B,POINT Z (415907.004 7113403.811 177.730),3
1,2,3.02,W12A,POINT Z (416506.021 7118415.894 0.000),1
2,3,2.79,W11C,POINT Z (416772.825 7118151.784 0.000),3
3,4,2.02,W14C,POINT Z (418134.709 7117944.425 0.000),3
4,5,3.83,W20E,POINT Z (421670.482 7117499.113 0.000),3
5,6,3.33,W23E,POINT Z (421978.128 7119306.260 0.000),3
6,7,3.06,W24A,POINT Z (423367.326 7119561.840 0.000),3
7,8,3.00,W01D,POINT Z (421542.589 7116016.646 0.000),3
8,9,3.63,EW01A,POINT Z (424243.862 7120024.173 0.000),3
9,10,0.88,EW09A,POINT Z (426228.780 7119372.952 0.000),3
